<a href="https://www.kaggle.com/code/aleksandrmorozov123/nlp-with-transformers?scriptVersionId=237265637" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/spanish-wine-quality-dataset/wines_SPA.csv
/kaggle/input/pakistan-law-data/pdf_data.json
/kaggle/input/vivino-brazilian-wines-metadata/brazilian_wine_data.json
/kaggle/input/vivino-brazilian-wines-metadata/brazilian_wine_data.csv
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1378590_10K_2021_0001437749-21-028984.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1408278_10K_2020_0001408278-21-000050.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1066194_10K_2021_0001558370-21-012388.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1348036_10K_2020_0001564590-21-008919.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1567900_10K_2020_0001437749-21-007800.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/1423774_10K_2021_0001423774-21-000020.json
/kaggle/input/sec-edgar-annual-financial-filings-2021/extracted/877890_10K_2020_0000877890-21-000051.json
/kaggle/

In [2]:
import numpy as np
import json
import re
import string
from IPython.display import display, HTML
import pandas as pd

import tensorflow as tf
from tensorflow.keras import layers, models, losses, callbacks

2025-05-01 18:04:51.971008: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-05-01 18:04:51.971157: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-01 18:04:52.135593: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


**Set the parameters of the model**

In [3]:
VOCAB_SIZE = 10000
MAX_LEN = 80
EMBEDDING_DIM = 256
KEY_DIM = 256
N_HEADS = 2
FEED_FORWARD_DIM = 256
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 5

**Loading the data**

In [4]:
# Load the full dataset
with open("/kaggle/input/wine-reviews/winemag-data-130k-v2.json") as json_data:
    wine_data = json.load(json_data)

In [5]:
# Filter the dataset
filtered_data = [
    "wine review : "
    + x["title"]
    + " : "
    + x["description"]
    + " : "
    + x["variety"]
    + " : "
    + x["province"]
    for x in wine_data
    if x ["title"] is not None
    and x["description"] is not None
    and x["variety"] is not None
    and x["province"] is not None
]

In [6]:
# count the reviews
n_wines = len (filtered_data)
print (f"{n_wines} reviews loaded")

129907 reviews loaded


In [7]:
example = filtered_data [20]
print (example)

wine review : Quiévremont 2012 Vin de Maison Red (Virginia) : Ripe aromas of dark berries mingle with ample notes of black pepper, toasted vanilla and dusty tobacco. The palate is oak-driven in nature, but notes of tart red currant shine through, offering a bit of levity. : Red Blend : Virginia


**Tokenize the data**

In [8]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}, '\n'])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s


text_data = [pad_punctuation(x) for x in filtered_data]

In [9]:
# example of the data
example_data = text_data[20]
example_data

'wine review : Quiévremont 2012 Vin de Maison Red ( Virginia ) : Ripe aromas of dark berries mingle with ample notes of black pepper , toasted vanilla and dusty tobacco . The palate is oak - driven in nature , but notes of tart red currant shine through , offering a bit of levity . : Red Blend : Virginia'

In [10]:
# Convert to a Tensorflow Dataset
text_ds = (
    tf.data.Dataset.from_tensor_slices(text_data)
    .batch(BATCH_SIZE)
    .shuffle(1000)
)

In [11]:
# Create a vectorisation layer
vectorize_layer = layers.TextVectorization(
    standardize="lower",
    max_tokens=VOCAB_SIZE,
    output_mode="int",
    output_sequence_length=MAX_LEN + 1,
)

In [12]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [13]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: :
3: ,
4: .
5: and
6: the
7: wine
8: a
9: of


**Create the training set**

In [14]:
# Create the training set and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y


train_ds = text_ds.map(prepare_inputs)

In [15]:
example_input_output = train_ds.take(1).get_single_element()

In [16]:
# Example Input
example_input_output[0][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([   7,   10,    2,    1,   69,   36,   33,   12,  215,   30,   12,
        193,   11,   11,    2,  948,   28,    9, 2315,    3,  325,    3,
        146,    5,   42,   46,  370,   50, 1641,   24,   19,    4,   17,
         18,   23,    8, 2982,  530,    9,    6,  515,    4,    2,   36,
         33,    2,  107,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

In [17]:
# Example Output (shifted by one token)
example_input_output[1][0]

<tf.Tensor: shape=(80,), dtype=int64, numpy=
array([  10,    2,    1,   69,   36,   33,   12,  215,   30,   12,  193,
         11,   11,    2,  948,   28,    9, 2315,    3,  325,    3,  146,
          5,   42,   46,  370,   50, 1641,   24,   19,    4,   17,   18,
         23,    8, 2982,  530,    9,    6,  515,    4,    2,   36,   33,
          2,  107,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0])>

**Casual attention mask function**

In [18]:
def causal_attention_mask(batch_size, n_dest, n_src, dtype):
    i = tf.range(n_dest)[:, None]
    j = tf.range(n_src)
    m = i >= j - n_src + n_dest
    mask = tf.cast(m, dtype)
    mask = tf.reshape(mask, [1, n_dest, n_src])
    mult = tf.concat(
        [tf.expand_dims(batch_size, -1), tf.constant([1, 1], dtype=tf.int32)], 0
    )
    return tf.tile(mask, mult)


np.transpose(causal_attention_mask(1, 10, 10, dtype=tf.int32)[0])

array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 1, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 1, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 1, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 1, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 1, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 1, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]], dtype=int32)

**Transformer block layer**

In [19]:
class TransformerBlock(layers.Layer):
    def __init__(self, num_heads, key_dim, embed_dim, ff_dim, dropout_rate=0.1):
        super(TransformerBlock, self).__init__()
        self.num_heads = num_heads
        self.key_dim = key_dim
        self.embed_dim = embed_dim
        self.ff_dim = ff_dim
        self.dropout_rate = dropout_rate
        self.attn = layers.MultiHeadAttention(
            num_heads, key_dim, output_shape=embed_dim
        )
        self.dropout_1 = layers.Dropout(self.dropout_rate)
        self.ln_1 = layers.LayerNormalization(epsilon=1e-6)
        self.ffn_1 = layers.Dense(self.ff_dim, activation="relu")
        self.ffn_2 = layers.Dense(self.embed_dim)
        self.dropout_2 = layers.Dropout(self.dropout_rate)
        self.ln_2 = layers.LayerNormalization(epsilon=1e-6)

    def call(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size = input_shape[0]
        seq_len = input_shape[1]
        causal_mask = causal_attention_mask(
            batch_size, seq_len, seq_len, tf.bool
        )
        attention_output, attention_scores = self.attn(
            inputs,
            inputs,
            attention_mask=causal_mask,
            return_attention_scores=True,
        )
        attention_output = self.dropout_1(attention_output)
        out1 = self.ln_1(inputs + attention_output)
        ffn_1 = self.ffn_1(out1)
        ffn_2 = self.ffn_2(ffn_1)
        ffn_output = self.dropout_2(ffn_2)
        return (self.ln_2(out1 + ffn_output), attention_scores)

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "key_dim": self.key_dim,
                "embed_dim": self.embed_dim,
                "num_heads": self.num_heads,
                "ff_dim": self.ff_dim,
                "dropout_rate": self.dropout_rate,
            }
        )
        return config

**Token and Position embedding**

In [20]:
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, max_len, vocab_size, embed_dim):
        super(TokenAndPositionEmbedding, self).__init__()
        self.max_len = max_len
        self.vocab_size = vocab_size
        self.embed_dim = embed_dim
        self.token_emb = layers.Embedding(
            input_dim=vocab_size, output_dim=embed_dim
        )
        self.pos_emb = layers.Embedding(input_dim=max_len, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "max_len": self.max_len,
                "vocab_size": self.vocab_size,
                "embed_dim": self.embed_dim,
            }
        )
        return config

**Build the Ttansformer model**

In [21]:
inputs = layers.Input(shape=(None,), dtype=tf.int32)
x = TokenAndPositionEmbedding(MAX_LEN, VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x, attention_scores = TransformerBlock(
    N_HEADS, KEY_DIM, EMBEDDING_DIM, FEED_FORWARD_DIM
)(x)
outputs = layers.Dense(VOCAB_SIZE, activation="softmax")(x)
gpt = models.Model(inputs=inputs, outputs=[outputs, attention_scores])
gpt.compile("adam", loss=[losses.SparseCategoricalCrossentropy(), None])

In [22]:
gpt.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, None)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ token_and_position_embedding    │ (None, None, 256)      │     2,580,480 │
│ (TokenAndPositionEmbedding)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ transformer_block               │ [(None, None, 256),    │       658,688 │
│ (TransformerBlock)              │ (None, 2, None, None)] │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, None, 10000)    │     2,570,000 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,809,168 (22.16 MB)

 Trainable params: 5,809,168 (22.16 MB)

 Non-trainable params: 0 (0.00 B)

**Train the Transformer model**

In [23]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
    def __init__(self, index_to_word, top_k=10):
        self.index_to_word = index_to_word
        self.word_to_index = {
            word: index for index, word in enumerate(index_to_word)
        }

    def sample_from(self, probs, temperature):
        probs = probs ** (1 / temperature)
        probs = probs / np.sum(probs)
        return np.random.choice(len(probs), p=probs), probs

    def generate(self, start_prompt, max_tokens, temperature):
        start_tokens = [
            self.word_to_index.get(x, 1) for x in start_prompt.split()
        ]
        sample_token = None
        info = []
        while len(start_tokens) < max_tokens and sample_token != 0:
            x = np.array([start_tokens])
            y, att = self.model.predict(x, verbose=0)
            sample_token, probs = self.sample_from(y[0][-1], temperature)
            info.append(
                {
                    "prompt": start_prompt,
                    "word_probs": probs,
                    "atts": att[0, :, -1, :],
                }
            )
            start_tokens.append(sample_token)
            start_prompt = start_prompt + " " + self.index_to_word[sample_token]
        print(f"\ngenerated text:\n{start_prompt}\n")
        return info

    def on_epoch_end(self, epoch, logs=None):
        self.generate("wine review", max_tokens=80, temperature=1.0)

In [24]:
# Create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath="./checkpoint/checkpoint.ckpt.weights.h5",
    save_weights_only=True,
    save_freq="epoch",
    verbose=0,
)

tensorboard_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [25]:
gpt.fit(
    train_ds,
    epochs=EPOCHS,
    callbacks=[model_checkpoint_callback, tensorboard_callback, text_generator],
)

Epoch 1/5
   3/4060 ━━━━━━━━━━━━━━━━━━━━ 2:05 31ms/step - loss: 9.0294  

I0000 00:00:1746122750.578927      69 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1746122750.591491      69 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


3668/4060 ━━━━━━━━━━━━━━━━━━━━ 8s 21ms/step - loss: 2.9799

W0000 00:00:1746122828.429212      70 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


4058/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.9364

W0000 00:00:1746122840.748622      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1746122864.670146      71 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update
W0000 00:00:1746122868.493684      72 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update



generated text:
wine review : franciscan 2015 chardonnay ( columbia valley ( wa ) ) : lightly aromatic with lighter than those pear fruit from apple , this is a light , fresh , watery wine with a depth and offers enjoyment of enjoyment and . it spent 14 months in bottle . : chardonnay : washington 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 129s 30ms/step - loss: 2.9361
Epoch 2/5
4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step - loss: 2.1726
generated text:
wine review : grassini 2006 sauvignon blanc ( happy canyon of santa barbara ) : a pungent nose of green banana , gooseberry , grapefruits and passion fruit on the palate , which is framed by brisk acidity , offering a splash in the mouth , which is just acidic . a little number to [UNK] it ' s riper and softer , more than expensive , but not too . : sauvignon blanc : california 

4060/4060 ━━━━━━━━━━━━━━━━━━━━ 100s 25ms/step - loss: 2.1726
Epoch 3/5
4060/4060 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 2.0855
generated text:
wine review : david hil

**Generate text using the Transformer**

In [26]:
def print_probs(info, vocab, top_k=5):
    for i in info:
        highlighted_text = []
        for word, att_score in zip(
            i["prompt"].split(), np.mean(i["atts"], axis=0)
        ):
            highlighted_text.append(
                '<span style="background-color:rgba(135,206,250,'
                + str(att_score / max(np.mean(i["atts"], axis=0)))
                + ');">'
                + word
                + "</span>"
            )
        highlighted_text = " ".join(highlighted_text)
        display(HTML(highlighted_text))

        word_probs = i["word_probs"]
        p_sorted = np.sort(word_probs)[::-1][:top_k]
        i_sorted = np.argsort(word_probs)[::-1][:top_k]
        for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
        print("--------\n")

In [27]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=1.0
)


generated text:
wine review: Toscana will grillo [UNK] 2010 [UNK] grillo ( sicilia ) : what doing ' re a flourish as well and [UNK] [UNK] farmed by the estate vineyard . this is a dense sauvignon , oily mouthfeel like stone on the palate , a [UNK] of crisp green melon and pineapple . it ' s enticing now , a nice choice cleanser . : grillo : sicily & sardinia 



In [28]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=0.5
)


generated text:
wine review: Toscana [UNK] ( [UNK] ) 2014 [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] [UNK] of [UNK] [UNK] [UNK] [UNK] . [UNK] [UNK] [UNK] [UNK] [UNK] ( [UNK] ) : [UNK] [UNK] with [UNK] [UNK] , ” but this blend of pinot nero and nerello mascalese opens with a [UNK] of [UNK] [UNK] . ” it ' s a [UNK] between fruit and [UNK] - [UNK] . [UNK] [UNK] , [UNK]



In [29]:
info = text_generator.generate(
    "wine review: Toscana", max_tokens=80, temperature=0.5
)
print_probs(info, vocab)


generated text:
wine review: Toscana [UNK] [UNK] [UNK] 2008 [UNK] [UNK] [UNK] ( amarone ) : a [UNK] , ” is a [UNK] of [UNK] , this wine , which is [UNK] [UNK] that [UNK] [UNK] the [UNK] . ” it has the aromas of baked bread , dark fruit , spice and leather . the mouthfeel is firm and tannic , with a tannic astringency . : [UNK] : [UNK] 



[UNK]:   	98.49%
review:   	1.28%
':   	0.11%
and:   	0.03%
.:   	0.01%
--------



[UNK]:   	91.6%
(:   	3.45%
red:   	2.66%
vineyard:   	0.7%
-:   	0.3%
--------



[UNK]:   	81.27%
2013:   	2.2%
2008:   	1.7%
2014:   	1.68%
2005:   	1.66%
--------



[UNK]:   	77.43%
(:   	2.26%
2013:   	2.14%
2008:   	2.08%
2014:   	1.8%
--------



...

::   	91.73%
it:   	2.45%
the:   	2.35%
[UNK]:   	1.09%
this:   	0.44%
--------



[UNK]:   	93.39%
red:   	4.17%
nebbiolo:   	1.74%
bordeaux:   	0.24%
gaglioppo:   	0.14%
--------



::   	98.86%
[UNK]:   	0.89%
and:   	0.06%
,:   	0.05%
is:   	0.02%
--------



[UNK]:   	99.94%
bordeaux:   	0.01%
burgundy:   	0.01%
in:   	0.01%
valpolicella:   	0.0%
--------



:   	100.0%
::   	0.0%
valley:   	0.0%
river:   	0.0%
familia:   	0.0%
--------

